In [1]:
from sys import stdout
import numpy as np
from mdtraj.reporters import HDF5Reporter
import openmm as mm
from openmm import app
from openmm import unit
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from openff.toolkit.topology import Molecule

## Load protein topology and positions

In [2]:
protein_file = "../2_receptor_preparation/fixed_receptor.pdb"
protein = app.PDBFile(protein_file)
modeller = app.Modeller(protein.topology, protein.positions)

## Load ligand topology and positions

In [3]:
ligand_file = "../3_ligand_preparation/ligand.sdf"
ligand = Molecule.from_file(ligand_file)
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()
modeller.add(ligand_topology.to_openmm(), ligand_positions)

## Load forcefield

In [4]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
forcefield.registerTemplateGenerator(smirnoff.generator)

## Add solvent

In [5]:
clearance = 14*unit.angstroms
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = mm.Vec3(1.0, 0, 0), mm.Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), mm.Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size + clearance)*v for v in vectors]

In [6]:
modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, neutralize=True)

/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/diego/Myopt/miniconda3/envs/uibcdf/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [7]:
# Verify that water and ions were added
n_waters = 0
ions = []
for chain in modeller.topology.chains():
    for residue in chain.residues():
        if residue.name=='HOH':
            n_waters += 1
        if len(list(residue.atoms()))==1:
            ions += [atom.name for atom in residue.atoms()]            

print('n_waters: {}'.format(n_waters))
print('ions: {}'.format(ions))

n_waters: 16777
ions: ['Na', 'Na', 'Na', 'Na']


## System creation

In [8]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer,
                                 constraints=app.HBonds, rigidWater=True)

for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        nonbondedForce = force

nonbondedForce.setUseDispersionCorrection(True)
nonbondedForce.setEwaldErrorTolerance(1.0e-5)
nonbondedForce.setUseSwitchingFunction(True)
nonbondedForce.setSwitchingDistance(0.8*unit.nanometer)

## Platform

In [9]:
platform = mm.Platform.getPlatformByName('CUDA')

## Integrator

In [10]:
integration_timestep = 2.0*unit.femtoseconds
temperature = 300.0*unit.kelvin

integrator = mm.LangevinIntegrator(temperature, 1.0/unit.picosecond, integration_timestep)

## Simulation object

In [11]:
simulation = app.Simulation(modeller.topology, system, integrator, platform)

## Initial coordinates

In [12]:
simulation.context.setPositions(modeller.positions)

## Energy minimization

In [13]:
# Check the potential energy before minimization
initial_state = simulation.context.getState(getEnergy=True)
initial_state.getPotentialEnergy()

Quantity(value=6687888992977919.0, unit=kilojoule/mole)

In [14]:
simulation.minimizeEnergy()

In [15]:
# Check the potential energy before minimization
minimized_state = simulation.context.getState(getEnergy=True)
minimized_state.getPotentialEnergy()

Quantity(value=-872593.5016476503, unit=kilojoule/mole)

In [16]:
# Writing out the minimized system
minimized_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, minimized_positions, open('initial.pdb', 'w'))

## Initial velocities

In [17]:
simulation.context.setVelocitiesToTemperature(temperature)

## Run a short simulation

In [18]:
simulation_time = 200.0*unit.picoseconds
reporting_time = 20.0*unit.picoseconds
saving_time = 10.0*unit.picoseconds

simulation_steps = int(simulation_time/integration_timestep)
reporting_steps = int(reporting_time/integration_timestep)
saving_steps = int(saving_time/integration_timestep)

In [19]:
states_reporter = app.StateDataReporter(stdout, reporting_steps, step=True, potentialEnergy=True, temperature=True,
                                    progress=True, totalSteps=simulation_steps, speed=True, remainingTime=True)
traj_reporter = HDF5Reporter('traj.h5', saving_steps, coordinates=True, time=True, cell=True, potentialEnergy=True,
                             kineticEnergy=True, temperature=True)
simulation.reporters+=[states_reporter, traj_reporter]

In [20]:
simulation.step(simulation_steps)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)","Time Remaining"
10.0%,10000,-705875.0016476503,300.64043300780764,0,0.0002758502960205078,--
20.0%,20000,-706383.5016476503,298.3657586506521,23.4,73.97811079025269,9:51
30.0%,30000,-706377.0016476503,301.09870441309175,23.4,147.99776458740234,8:37
40.0%,40000,-706529.7516476503,301.46726031118476,23.3,222.0157310962677,7:24
50.0%,50000,-705871.0016476503,300.7028839567023,23.3,296.10841250419617,6:10
60.0%,60000,-706189.0016476503,299.18900176075147,23.3,370.20928049087524,4:56
70.0%,70000,-707239.5016476503,302.1875615764097,23.3,444.3628659248352,3:42
80.0%,80000,-704727.2516476503,300.0130898382372,23.3,518.5115087032318,2:28
90.0%,90000,-706924.0016476503,300.1361681894527,23.3,592.6566951274872,1:14
100.0%,100000,-705999.0016476503,301.2183760288581,23.3,666.8446862697601,0:00


In [21]:
traj_reporter.close()

In [22]:
# Writing out the final system
final_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, final_positions, open('final.pdb', 'w'))